# (15.1) Установка необходимых пакетов

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

# (15.2) Определение функций

# (15.3) Список всех использующихся датафреймов¶¶

# (15.4) Чтение данных

In [4]:
# создаем список из инструментов, относящихся к числу голубых фишек
tickers = ['GAZP',
           'SBER',
           'GMKN',
           'LKOH',
           'MTSS',
           'MGNT',
           'TATN',
           'NVTK',
           'YNDX',
           'ROSN',
           'FIVE',
           'VTBR',
           'SNGS',
           'CHMF',
           'ALRS'
          ]

In [254]:
# создаем список из всех названий файлов с данными
orderlogs = ['OrderLog20190304.txt',
             'OrderLog20190305.txt',
             'OrderLog20190306.txt',
             'OrderLog20190307.txt',
             'OrderLog20190311.txt',
             'OrderLog20190312.txt',
             'OrderLog20190313.txt',
             'OrderLog20190314.txt',
             'OrderLog20190315.txt',
             'OrderLog20190318.txt',
             'OrderLog20190319.txt',
             'OrderLog20190320.txt',
             'OrderLog20190321.txt',
             'OrderLog20190322.txt',
             'OrderLog20190325.txt',
             'OrderLog20190326.txt',
             'OrderLog20190327.txt',
             'OrderLog20190328.txt',
             'OrderLog20190329.txt'
            ]

In [255]:
# читаем файл с выявленными айсбергами
# iceberg_all - список всех айсбергов с указанием цепочки
iceberg_all = pd.read_csv('iceberg_new.csv', header = 0)
del iceberg_all['index']
iceberg_all.reset_index(level=0, inplace=True)
iceberg_all.rename(columns = {'index':'ID'}, inplace=True)

iceberg_all

FileNotFoundError: [Errno 2] File b'iceberg_new.csv' does not exist: b'iceberg_new.csv'

In [38]:
iceberg_all.dtypes

index             int64
BUYSELL          object
DATE              int64
ICEBERG           int64
ORDERNO           int64
PRICE           float64
SECCODE          object
TIME              int64
TIME_IDENT        int64
VOLUME            int64
VOLUME_INI        int64
GROUP_ID        float64
TOTAL_VOLUME    float64
dtype: object

In [56]:
# устанавливаем маску участия в цепочке (True - входит в цепочку)
chain = iceberg_all['GROUP_ID'].notnull()
# chained - айсберги, которые были объединены в цепочки
chained = iceberg_all[['DATE','ORDERNO','TOTAL_VOLUME','GROUP_ID','ID']][(chain == True)]
# not_chained - айсберги, которые не были объединены в цепочки
not_chained = iceberg_all[['DATE','ORDERNO','VOLUME','VOLUME_INI','ID']][(chain == False)]

In [253]:
%%time
# если айсберг-заявка не была объединена в цепочку, то для обновления ордерлога необходимо просто по 
# номеру заявки-дате найти соответствующую заявку с action = 1 в ордерлоге и обновить объем
# на величину (VOLUME_INI - VOLUME)
for l in orderlogs:

    date = int(l[8:16])
    orders = pd.read_csv(l, header = 0)
    icebergs = not_chained[not_chained.DATE == date].reset_index()
    
    for i in range(len(icebergs)):
        
        orderno = icebergs.loc[i,'ORDERNO']
        orders.loc[(orders['ORDERNO']==orderno) & (orders['ACTION']==1),'VOLUME'] -= icebergs.loc[i,'VOLUME']
        
    orderlog_name = '/Users/ekaterina/Desktop/w/курсовая (2)/data/updated/' + l       
    orders.to_csv(orderlog_name,index = False)

CPU times: user 20min 31s, sys: 2min 24s, total: 22min 55s
Wall time: 17min 52s


In [244]:
orders[orders.ORDERNO == 199077]

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
342352,342353,GAZP,B,100643595004,199077,1,156.28,4830,NaN,NaN
394216,394217,GAZP,B,100935807572,199077,2,156.28,550,2.935640e+09,156.28
394229,394230,GAZP,B,100935808904,199077,2,156.28,500,2.935640e+09,156.28
394266,394267,GAZP,B,100935902211,199077,2,156.28,10,2.935640e+09,156.28
394270,394271,GAZP,B,100935906230,199077,2,156.28,760,2.935640e+09,156.28
394341,394342,GAZP,B,100936427550,199077,2,156.28,10,2.935640e+09,156.28
394358,394359,GAZP,B,100936467866,199077,2,156.28,3000,2.935640e+09,156.28


In [236]:
test1 = not_chained[not_chained.DATE == 20190304].reset_index()
test1.loc[0,'ORDERNO']

199077

In [235]:
test3 = not_chained[not_chained.ORDERNO == test1.loc[0,'ORDERNO']]
test3

,DATE,ORDERNO,VOLUME,VOLUME_INI,ID
0,20190304,199077,-2830,2000,0


In [234]:
test = pd.read_csv('OrderLog20190304.txt', header = 0)
test2 = test[(test.ORDERNO == test1.loc[0,'ORDERNO'])]
test2

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
342352,342353,GAZP,B,100643595004,199077,1,156.28,2000,NaN,NaN
394216,394217,GAZP,B,100935807572,199077,2,156.28,550,2.935640e+09,156.28
394229,394230,GAZP,B,100935808904,199077,2,156.28,500,2.935640e+09,156.28
394266,394267,GAZP,B,100935902211,199077,2,156.28,10,2.935640e+09,156.28
394270,394271,GAZP,B,100935906230,199077,2,156.28,760,2.935640e+09,156.28
394341,394342,GAZP,B,100936427550,199077,2,156.28,10,2.935640e+09,156.28
394358,394359,GAZP,B,100936467866,199077,2,156.28,3000,2.935640e+09,156.28


In [256]:
%%time
# если айсберг-заявка была объединена в цепочку, то для обновления ордерлога необходимо обновить первоначальный объем первой заявки в цепочке,
# удалить запись о выставлении
for l in orderlogs:

    date = int(l[8:16])
    orderlog_name = '/Users/ekaterina/Desktop/w/курсовая (2)/data/updated/' + l
    orders = pd.read_csv(orderlog_name, header = 0)
    icebergs = chained[chained.DATE == date].reset_index()
    
    for i in range(len(icebergs)):
        
        orderno = icebergs.loc[i,'ORDERNO']
        
        if icebergs.loc[i,'ID'] == icebergs.loc[i,'GROUP_ID']: # если это первый элемент цепочки айсберга, то ему необходимо перезаписать объем на объем цепочки
            
            first_element_id = icebergs.loc[i,'ORDERNO']
            orders.loc[(orders['ORDERNO']==orderno) & (orders['ACTION']==1),'VOLUME'] = icebergs.loc[i,'TOTAL_VOLUME']
            
        else: # если это не первый элемент цепочки айсберга, то ему необходимо:
            
            orders = orders.drop(orders[(orders['ORDERNO']==orderno) & (orders['ACTION']==1)].index) # удалить выставленные заявки
            orders.loc[(orders['ORDERNO']==orderno) & (orders['ACTION']!=1),'ORDERNO'] = first_element_id # поменять номер заявки сделки и отмены на номер первой заявки в цепочке
               
    orders.to_csv(orderlog_name,index = False)

KeyboardInterrupt: 

In [160]:
orders

,level_0,index,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,0,1821404,1821405,GAZP,B,120642051496,914063,1,155.99,1000,NaN,NaN
1,1,1821405,1821406,GAZP,B,120642051496,914063,2,155.99,4000,2.935752e+09,155.93
3,3,1821408,1821409,GAZP,B,120642051496,914064,2,155.99,1750,2.935752e+09,155.94
